In [ ]:
import numpy as np

In [ ]:
cd 'drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!ls

'19CS30006 (1).pdf'
'19CS30006 (2).pdf'
'19CS30006 (3).pdf'
 19CS30006_Aryan_Mehta.pdf
'19CS30006_FLAT_LT2 .pdf'
'19CS30006_G09_WS7.pdf (1).gdoc'
 19CS30006_LT1.pdf
 19CS30006_LT2.pdf
'19CS30006_LT3 (1).pdf'
 19CS30006_LT3.pdf
'19CS30006 .pdf'
 19CS30006.pdf
 19CS30006_Q2.pdf
 19CS30006_Q3.pdf
 19CS30006_Signet_Test_4.pdf
 19CS30006_UML.gdoc
 4th_Aryan_Mehta_19CS30006.gdoc
'5th Week Assignment-GNH.docx'
'5th Week Assignment-GNH.gdoc'
 6bitBCD.xopp
 6th_Aryan_Mehta_19CS30006.docx
 6th_Aryan_Mehta_19CS30006.gdoc
 7th_Aryan_Mehta_19CS30006.gdoc
 8th_Aryan_Mehta_19CS30006.gdoc
 ActivityDiagram.drawio
 AGV
'agv documentation 2020.gdoc'
 agv_obstacle_detection
 AGV_TASKS
'annotated DL research papers '
 Audio-Classification
 audio_data.csv
'basic ros commands.gdoc'
'b Club.gdoc'
 cache
 checkpoints_VGG_0.01
 Class_Diagram.drawio
 Classroom
'Colab Notebooks'
 CommunicationDiagram.drawio
'Copy of Abstract Origami Template – by PresentationGO.gslides'
'Copy of Gaoler · SlidesCarnival.gslides'
'

In [ ]:
#!unzip './shapenet-chairs-pcd.zip'

In [ ]:
def create_input_points_array(file_location):
    points=[]
    with open(file_location) as myfile:
        lines = myfile.readlines()[10:]
    for line in lines:
        coordinates = line.split(' ')
        coordinates=np.array(coordinates,dtype=np.float)
        points.append(coordinates)
    points = np.vstack(points)
    return points
    

KD-TREE BUILDING

In [ ]:
import os
from sklearn.neighbors import KDTree
base_file_location = 'shapenet-chairs-pcd/'
P=[]
for myfile in os.listdir(base_file_location):
    file_location = base_file_location + myfile
    points = create_input_points_array(file_location)
    #print(points)
    kd_tree = KDTree(points)
    sorted_points = points[kd_tree.get_arrays()[1]]
    P.append(sorted_points.flatten())      
P = np.vstack(P)
print(P.shape)    

(5000, 3000)


PCA ANALYSIS

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(100)
pca.fit(P)
U = pca.components_
V = pca.transform(P)
print(U.shape)
print(V.shape)
          

(100, 3000)
(5000, 100)


Interative Point Ordering


In [ ]:
iterations = 1000
K = 10000
for i in range(iterations):
    pca.fit(P)
    for j in range(K):
        p1,p2=np.random.randint(0,1000,2)
        shape=np.random.randint(0,5000,1)
        initial_loss=((pca.inverse_transform(pca.transform(P[shape]))-P[shape])**2).sum()
        swapped_loss = ((pca.inverse_transform(pca.transform(P[shape]))-P[shape])**2).sum()

        if swapped_loss > initial_loss:
            P[shape,3*p1 : 3*p1 +3] = P[shape,3*p2 : 3*p2 +3]
            P[shape,3*p2 : 3*p2 +3]= P[shape,3*p1 : 3*p1 +3]



GAN Training


In [ ]:
from torch import nn,optim
from torch.nn import functional as F
from torchvision import datasets,models,transforms


generator=nn.Sequential(nn.Linear(100,100),
                      nn.BatchNorm1d(100),
                      nn.ReLU(),
                      nn.Linear(100,100),
                      nn.BatchNorm1d(100),
                      nn.ReLU(),
                      nn.Linear(100,100),
                      nn.BatchNorm1d(100),
                      nn.ReLU(),
                      nn.Linear(100,100)).cuda()

discriminator=nn.Sequential(nn.Linear(100,100),
                      nn.BatchNorm1d(100),
                      nn.LeakyReLU(),
                      nn.Linear(100,100),
                      nn.BatchNorm1d(100),
                      nn.LeakyReLU(),
                      nn.Linear(100,100),
                      nn.BatchNorm1d(100),
                      nn.LeakyReLU(),
                      nn.Linear(100,1)).cuda()

optimizer_discriminator = optim.Adam(discrimator.parameters(),lr=0.0001)
optimizer_generator = optim.Adam(generator.parameters(),lr=0.0025)

RuntimeError: ignored

In [ ]:
for epoch in range(epochs):
    optimizer_discriminator.zero_grad()
    optimizer_generator.zero_grad()

    fake_data = generator(torch.rand(5000,100))
    real_data = torch.tensor(pca.transform(P),dtype=torch.float,device='cuda')
 
    discriminator_loss = (discriminator(real)).mean() + (1-discriminator(fake)).mean()

    intermediate_activations_fake = discriminator[:-1](fake)
    intermediate_activations_real = discriminator[:-1](real)

    generator_loss = ((intermediate_activations_fake-intermediate_activations_real)**2).mean() + ((cov(intermediate_activations_fake)-cov(intermediate_activations_real))**2).mean()

    discriminator_loss.backward()
    optimizer_discriminator.step()

    generator_loss.backward()
    optimizer_generator.step()